In [1]:
import pandas as pd

In [2]:
article_sample = pd.read_csv("./data/articles_sample.csv")

In [3]:
system_prompt = "Sei un esperto nella valutazione della credibilità dei siti di notizie. Il tuo compito è rispondere a una serie di domande progettate per analizzare e valutare la qualità del testo di un articolo giornalistico, offrendo un giudizio competente e imparziale."
qIDs = ["Type", "HeadAcc1", "HeadAcc2", "LedePres", "ArtBias", "SensLang", "NegTarg1", "NegTarg2"]

prompts_WO_json_by_qID: dict = {
    "Type" : "Ecco un articolo di giornale dal titolo %...% ed il seguente testo %...%. \nChe tipo di notizia hai letto? Seleziona una tra seguenti opzioni e fornisci una spiegazione concisa della tua scelta:\n 1 Notizia di cronaca, attualita’, notizia che riguarda il paese o notizia internazionale\n 2 Editoriale o opinione\n 3 Inchiesta\n 4 Satira\n 5 Notizia di costume o cronaca rosa.",
    "HeadAcc1" : "Ecco un articolo di giornale dal titolo %...% ed il seguente testo %...%. \nDimmi quanto è accurato il titolo della notizia rispetto al contenuto e forniscimi una spiegazione concisa della tua scelta:\n  1. Non accurato\n  2. Abbastanza inaccurato\n  3. Abbastanza accurato\n  4. Accurato",
    "LedePres" : "Ecco un articolo di giornale dal titolo %...% ed il seguente testo %...%. \nDimmi se l'articolo inizia con un riassunto del fatto principale e forniscimi una spiegazione concisa della tua scelta:\n    1. No\n    2. Si",
    "ArtBias" : "Ecco un articolo di giornale dal titolo %...% ed il seguente testo %...%. \nDimmi quanto l'articolo è di parte e forniscimi una spiegazione concisa della tua scelta:\n    1. Di parte\n    2. Abbastanza di parte\n    3. Abbastanza imparziale\n    4. Imparziale",
    "SensLang" : "Ecco un articolo di giornale dal titolo %...% ed il seguente testo %...%. \nDimmi quanto è sensazionale il tono della notizia e forniscimi una spiegazione concisa della tua scelta:\n    1. Sensazionale\n    2. Abbastanza sensazionale\n    3. Abbastanza neutro\n    4. Neutro",
    "NegTarg1" : "Ecco un articolo di giornale dal titolo %...% ed il seguente testo %...%. \nDimmi se l'articolo prende di mira negativamente individui o gruppi e forniscimi una spiegazione concisa della tua scelta:\n    1. No\n    2. Si",
    "NegTarg2" : "Ecco un articolo di giornale dal titolo %...% ed il seguente testo %...%. \nSe l'articolo prende di mira negativamente individui o gruppi dimmi su quale tematica sono basati e forniscimi una spiegazione concisa della tua scelta:\n    1. Politica\n    2. Gender\n    3. Religione\n    4. Altro"
}

questions_by_qID:dict = {
    "Type" : "Che tipo di notizia hai letto?",
    "HeadAcc1" : "Quanto è accurato il titolo della notizia rispetto al contenuto?",
    "LedePres" : "L'articolo inizia con un riassunto del fatto principale?",
    "ArtBias" : "L'articolo è di parte?",
    "SensLang" : "Quanto è sensazionale il tono della notizia?",
    "NegTarg1" : "L'articolo prende di mira negativamente individui o gruppi?",
    "NegTarg2" : "Se l'articolo prende di mira negativamente individui o gruppi dimmi su quale tematica sono basati?"
}

responses_by_qID:dict = {
    "Type" : {"1" : "Notizia di cronaca, attualita’, notizia che riguarda il paese o notizia internazionale", "2" : "Editoriale o opinione", "3" : "Inchiesta", "4" : "Satira", "5" : "Notizia di costume o cronaca rosa"},
    "HeadAcc1" : {"1" : "Non accurato", "2" : "Abbastanza inaccurato", "3" : "Abbastanza accurato", "4" : "Accurato"},
    "HeadAcc2" : {"1" : "Una domanda", "2" : "Pronomi dimostrativi", "3" : "Termini scritti in maiuscolo", "4" : "Superlativi (come il più alto, altissimo..) o intensificatori (come molto, estremamente, veramente..)"},
    "LedePres" : {"1" : "No", "2" : "Si"},
    "ArtBias" : {"1" : "Di parte", "2" : "Abbastanza di parte", "3" : "Abbastanza imparziale", "4" : "Imparziale"},
    "SensLang" : {"1" : "Sensazionale", "2" : "Abbastanza sensazionale", "3" : "Abbastanza neutro", "4" : "Neutro"},
    "NegTarg1" : {"1" : "No", "2" : "Si"},
    "NegTarg2" : {"1" : "Politica", "2" : "Gender", "3" : "Religione", "4" : "Altro"}
}


# A simple Example 

In [4]:
example_article_title = article_sample.iloc[0]["title"]
example_article_text = article_sample.iloc[0]["text"]

example_type_criteria_user_prompt = prompts_WO_json_by_qID[ qIDs[0] ]

example_type_criteria_user_prompt = example_type_criteria_user_prompt.replace('%...%', f'"{example_article_title}"', 1)
example_type_criteria_user_prompt = example_type_criteria_user_prompt.replace('%...%', f'"{example_article_text}"', 1)

print( example_type_criteria_user_prompt )

Ecco un articolo di giornale dal titolo "«Fare meno tamponi vuol dire rinunciare a tracciare il Covid»" ed il seguente testo "Nei prossimi giorni il governo presenterà il nuovo sistema di monitoraggio con cui vigilare sulle riaperture. Il premier Draghi ha mostrato prudenza, ma le regioni premono affinché i “colori” del rischio vengano assegnati con criteri diversi da quelli usati finora. In particolare, chiedono di archiviare l’indice Rt, di non valutare le attività di tracciamento e di fissare un numero di test più basso di quello attuale. Il microbiologo Andrea Crisanti, che nel 2020 aveva guidato la strategia di test del Veneto unanimemente ritenuta un modello, aveva proposto di arrivare a 500 mila test al giorno per aumentare la sorveglianza sul virus. Invece, secondo le regioni, con l’attuale incidenza del virus invece dei 287 mila test effettuati ieri ne basterebbero meno di novantamila.

Dottor Crisanti, immaginiamo che lei non sia d’accordo.

Significa rinunciare totalmente al

# GPT API Settings

In [5]:
"""
AZURE_OPENAI_KEY="XXX"

client = AzureOpenAI(
    azure_endpoint="https://chatgpt4asannotator.openai.azure.com/openai/deployments/gpt-4o_MarinellaPetrocchi/chat/completions?api-version=2024-02-15-preview",
    api_key=AZURE_OPENAI_KEY,
    api_version="2024-02-15-preview",
)
messages = [{
                "role": "system",
                "content": str(system_prompt)
            },
            {
                "role": "user",
                "content": str(example_type_criteria_user_prompt)
            }
    ]

completion = client.chat.completions.create(
                ...
                messages = messages,
                # The temperature is zero because it concerns the degree of creativity of the messages but our outputs are numbers so we don't need it, in fact increasing it would risk having an invalid JSON produced
                temperature=0,
                # Maximum Length either because the texts of the articles vary.
                # This is also about creativity and we need it low so I'll throw it in at a minimum
                top_p=0.01,
                # frequence and presence are used to avoid repetition but we don't care so they are left to default to 0
                frequency_penalty=0,
                presence_penalty=0,
                # Stop Sequences we don't need it because we don't have an end sequence beyond which it should not generate so I give it “None”
                stop=None
            )

resp = completion.choices[0].message.content
"""

'\nAZURE_OPENAI_KEY="XXX"\n\nclient = AzureOpenAI(\n    azure_endpoint="https://chatgpt4asannotator.openai.azure.com/openai/deployments/gpt-4o_MarinellaPetrocchi/chat/completions?api-version=2024-02-15-preview",\n    api_key=AZURE_OPENAI_KEY,\n    api_version="2024-02-15-preview",\n)\nmessages = [{\n                "role": "system",\n                "content": str(system_prompt)\n            },\n            {\n                "role": "user",\n                "content": str(example_type_criteria_user_prompt)\n            }\n    ]\n\ncompletion = client.chat.completions.create(\n                ...\n                messages = messages,\n                # The temperature is zero because it concerns the degree of creativity of the messages but our outputs are numbers so we don\'t need it, in fact increasing it would risk having an invalid JSON produced\n                temperature=0,\n                # Maximum Length either because the texts of the articles vary.\n                # This is